In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import logging
from transformers import pipeline
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
from PipelineCacheWrapper.PipelineCacheWrapper import PipelineCacheWrapper

In [2]:
from TRExData.LamaTRExData import LamaTRExData
from SentenceTypologyQueryResults import SentenceTypologyQueryResults
from relation_templates.templates import relations, nominalized_relations

In [3]:
from TypologyQuerier import TypologyQuerier 

In [4]:
logging.set_verbosity_error()

In [5]:
model = "google/electra-large-generator"

In [6]:
MASK = "[MASK]"
KEYS = ["active", "passive", "nominalized"]

In [7]:
#relations = ["P19", "P36", "P101", "P103","P106","P108", "P178", "P1001"]
#relations = ["P19", "P413", "P159", "P103"]
#relations = ["P364"]
#relations = nominalized_relations
#relations = ['P138', 'P276', 'P31', 'P106', 'P413', 'P937', 'P1303', 'P127', 'P279', 'P527', 'P136', 'P39', 'P101', 'P361', 'P463', 'P176']
#relations =['P138', 'P276', 'P31', 'P937', 'P279', 'P527', 'P101', 'P361']

In [8]:
TOP_K = 1

In [9]:
#unmasker = pipeline('fill-mask', model='bert-large-uncased-whole-word-masking', top_k=100)
unmasker = PipelineCacheWrapper('fill-mask', model=model, top_k=100)
#unmasker = pipeline("fill-mask", model="google/electra-large-generator", tokenizer="google/electra-large-generator", top_k=TOP_K) 

In [10]:
TREx = LamaTRExData(relations = relations)
TREx.load()
#TREx.load_abstract()

In [11]:
querier = TypologyQuerier(unmasker, relations, TOP_K, MASK)

In [12]:
#querier.query(TREx.abstract_data)
querier.query(TREx.data)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 982/982 [01:37<00:00, 10.07it/s]


In [13]:
%%time
unmasker.save_to_cache()

CPU times: user 8.7 s, sys: 691 ms, total: 9.4 s
Wall time: 9.8 s


In [14]:
querier.print_global_result()

total Result
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |       635 |   34039 |   1.86551  |
| compound         |       230 |   34039 |   0.675696 |
| complex          |       413 |   34039 |   1.21331  |
| compound-complex |        90 |   34039 |   0.264403 |
Total Result for 1:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |         0 |     937 |          0 |
| compound         |         0 |     937 |          0 |
| complex          |         0 |     937 |          0 |
| compound-complex |         0 |     937 |          0 |
Total Result for N:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |       459 |   20928 |   2.19323  |
| compound         |       164 |   20928 |   0.783639 |
| complex          |       316 |   20928 |   1.50

In [15]:
querier.print_global_latex()

Total  & 1.87 & 0.68 & 1.21 & 0.26 \\
1:1  & 0.00 & 0.00 & 0.00 & 0.00 \\
N:1  & 2.19 & 0.78 & 1.51 & 0.22 \\
N:M  & 1.45 & 0.54 & 0.80 & 0.36 \\


In [16]:
querier.print_for_latex()

P1376 & capital of & 1:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P36 & capital & 1:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P103 & native language & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P127 & owned by & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P131 & located in the administrative territorial entity & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P136 & genre & N:1 & 0.54 & 2.58 & 16.00 & 1.40 \\
P138 & named after & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P140 & religion or worldview & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P159 & headquarters location & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P17 & country & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P176 & manufacturer & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P19 & place of birth & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P20 & place of death & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P264 & record label & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P276 & location & N:1 & 0.10 & 0.00 & 0.00 & 0.00 \\
P279 & subclass of & N:1 & 22.30 & 13.17 & 3.94 & 2.49 \\
P30 & continent & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P31

In [17]:
TOP_K = 10

In [18]:
querier.reset()
querier.set_top_k(TOP_K)

In [19]:
querier.query(TREx.data)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 982/982 [00:00<00:00, 33062.87it/s]


In [20]:
querier.print_global_result()

total Result
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |      1852 |   34039 |    5.44082 |
| compound         |      1472 |   34039 |    4.32445 |
| complex          |      1548 |   34039 |    4.54772 |
| compound-complex |      1474 |   34039 |    4.33033 |
Total Result for 1:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |         0 |     937 |          0 |
| compound         |         0 |     937 |          0 |
| complex          |         0 |     937 |          0 |
| compound-complex |         0 |     937 |          0 |
Total Result for N:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |      1086 |   20928 |    5.18922 |
| compound         |       753 |   20928 |    3.59805 |
| complex          |       967 |   20928 |    4.6

In [21]:
querier.print_global_latex()

Total  & 5.44 & 4.32 & 4.55 & 4.33 \\
1:1  & 0.00 & 0.00 & 0.00 & 0.00 \\
N:1  & 5.19 & 3.60 & 4.62 & 3.79 \\
N:M  & 6.29 & 5.91 & 4.77 & 5.59 \\


In [22]:
querier.print_for_latex()

P1376 & capital of & 1:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P36 & capital & 1:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P103 & native language & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P127 & owned by & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P131 & located in the administrative territorial entity & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P136 & genre & N:1 & 13.32 & 21.48 & 52.42 & 6.12 \\
P138 & named after & N:1 & 0.62 & 0.62 & 0.31 & 0.78 \\
P140 & religion or worldview & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P159 & headquarters location & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P17 & country & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P176 & manufacturer & N:1 & 0.00 & 0.10 & 0.00 & 0.00 \\
P19 & place of birth & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P20 & place of death & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P264 & record label & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P276 & location & N:1 & 0.10 & 0.00 & 0.00 & 0.00 \\
P279 & subclass of & N:1 & 48.13 & 46.27 & 14.11 & 43.57 \\
P30 & continent & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\

In [23]:
TOP_K = 100

In [24]:
querier.reset()
querier.set_top_k(TOP_K)

In [25]:
querier.query(TREx.data)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 982/982 [00:00<00:00, 15187.84it/s]


In [26]:
querier.print_global_result()

total Result
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |      5127 |   34039 |   15.0621  |
| compound         |      3311 |   34039 |    9.72708 |
| complex          |      3195 |   34039 |    9.38629 |
| compound-complex |      3166 |   34039 |    9.3011  |
Total Result for 1:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |         0 |     937 |          0 |
| compound         |         0 |     937 |          0 |
| complex          |         0 |     937 |          0 |
| compound-complex |         0 |     937 |          0 |
Total Result for N:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |      2429 |   20928 |   11.6065  |
| compound         |      1783 |   20928 |    8.51969 |
| complex          |      1731 |   20928 |    8.2

In [27]:
querier.print_global_latex()

Total  & 15.06 & 9.73 & 9.39 & 9.30 \\
1:1  & 0.00 & 0.00 & 0.00 & 0.00 \\
N:1  & 11.61 & 8.52 & 8.27 & 8.40 \\
N:M  & 22.16 & 12.55 & 12.03 & 11.57 \\


In [28]:
querier.print_for_latex()

P1376 & capital of & 1:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P36 & capital & 1:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P103 & native language & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P127 & owned by & N:1 & 0.15 & 0.00 & 0.15 & 0.15 \\
P131 & located in the administrative territorial entity & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P136 & genre & N:1 & 74.11 & 74.65 & 82.28 & 35.77 \\
P138 & named after & N:1 & 1.24 & 1.40 & 1.09 & 1.40 \\
P140 & religion or worldview & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P159 & headquarters location & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P17 & country & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P176 & manufacturer & N:1 & 0.10 & 0.10 & 0.00 & 0.10 \\
P19 & place of birth & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P20 & place of death & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P264 & record label & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \\
P276 & location & N:1 & 0.10 & 0.00 & 0.10 & 0.00 \\
P279 & subclass of & N:1 & 69.50 & 66.08 & 36.00 & 63.49 \\
P30 & continent & N:1 & 0.00 & 0.00 & 0.00 & 0.00 \